In [1]:
import numpy as np
from jaqs.data import DataView
from jaqs.research import SignalDigger
import warnings

warnings.filterwarnings("ignore")
dataview_folder = 'stockdata/hs300'
dv = DataView()
dv.load_dataview(dataview_folder)

/home/xinger/anaconda3/envs/IIA/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


Dataview loaded successfully.


In [2]:
def mask_index_member():
    df_index_member = dv.get_ts('index_member')
    mask_index_member = ~(df_index_member >0) #定义信号过滤条件-非指数成分
    return mask_index_member

def limit_up_down():
    # 定义可买卖条件——未停牌、未涨跌停
    trade_status = dv.get_ts('trade_status')
    mask_sus = trade_status == u'停牌'
    # 涨停
    dv.add_formula('up_limit', '(close - Delay(close, 1)) / Delay(close, 1) > 0.095', is_quarterly=False)
    # 跌停
    dv.add_formula('down_limit', '(close - Delay(close, 1)) / Delay(close, 1) < -0.095', is_quarterly=False)
    can_enter = np.logical_and(dv.get_ts('up_limit') < 1, ~mask_sus) # 未涨停未停牌
    can_exit = np.logical_and(dv.get_ts('down_limit') < 1, ~mask_sus) # 未跌停未停牌
    return can_enter,can_exit

In [3]:
mask = mask_index_member()
can_enter,can_exit = limit_up_down()

In [4]:
from jaqs.research import Optimizer

In [5]:
# 优化因子
price = dv.get_ts('close_adj')
high = dv.get_ts('high_adj')
low = dv.get_ts('low_adj')
price_bench = dv.data_benchmark
optimizer = Optimizer(dataview=dv,
                      formula='- Correlation(vwap_adj, volume, LEN)',
                      params={"LEN":range(2,4,1)},
                      name='divert',
                      price=price,
                      high=high,
                      low=low,
                      benchmark_price=None,#=None求绝对收益 #=price_bench求相对收益
                      period=5,
                      n_quantiles=5,
                      mask=mask,
                      can_enter=can_enter,
                      can_exit=can_exit,
                      commission=0.0008,#手续费 默认0.0008
                      is_event=False,#是否是事件(0/1因子)
                      is_quarterly=False)#是否是季度因子 默认为False

### target_type:
* ic类:
  return_ic/upside_ret_ic/downside_ret_ic
* 持有收益类
  long_ret/short_ret/long_short_ret/top_quantile_ret/bottom_quantile_ret/tmb_ret
* 收益空间类
  long_space/short_space/long_short_space/top_quantile_space/bottom_quantile_space/tmb_space

### target:
* ic类 
  "IC Mean", "IC Std.", "t-stat(IC)", "p-value(IC)", "IC Skew", "IC Kurtosis", "Ann. IR"
* 持有收益类 
  't-stat', "p-value", "skewness", "kurtosis", "Ann. Ret", "Ann. Vol", "Ann. IR", "occurance"
* 收益空间类的'upside_space_mean','upside_space_std','upside_space_mean/std','upside_space_max','upside_space_min','upside_space_percentile25','upside_space_percentile50','upside_space_percentile75','upside_space_occurance','downside_space_mean','downside_space_std','downside_space_mean/std','downside_space_max',downside_space_min','downside_space_percentile25','downside_space_percentile50','downside_space_percentile75','downside_space_occurance','up&down_space_mean_sum',　    

In [6]:
ret_best = optimizer.enumerate_optimizer(target_type="long_ret",#优化目标类型 
                                         target="Ann. IR",#优化目标     
                                         in_sample_range=[20170101,20170701],#样本内范围 默认为None,在全样本上优化
                                         ascending=False)#是否按优化目标升序排列(从小到大)

Nan Data Count (should be zero) : 0;  Percentage of effective data: 67%
Nan Data Count (should be zero) : 0;  Percentage of effective data: 68%


In [7]:
print(ret_best[0]["signal_name"])
print(ret_best[0]["ret"])
print(ret_best[0]["ic"])
print(ret_best[0]["space"])

divert{'LEN': 3}
             long_ret   short_ret  long_short_ret  top_quantile_ret  \
t-stat       1.651698    0.127259        0.551727          6.409743   
p-value      0.101250    0.898950        0.582180          0.000000   
skewness    -0.347678    0.347192        0.234277          0.684339   
kurtosis    -0.297840    0.438304        2.124405         14.505309   
Ann. Ret     0.114892    0.009147        0.016518          0.148677   
Ann. Vol     0.108612    0.112229        0.046747          0.272277   
Ann. IR      1.057822    0.081502        0.353350          0.546051   
occurance  119.000000  119.000000      119.000000       6670.000000   

           bottom_quantile_ret     tmb_ret  
t-stat               -0.075431    3.432355  
p-value               0.939870    0.000830  
skewness              1.577147   -0.159136  
kurtosis             26.451335    0.969194  
Ann. Ret             -0.001845    0.149774  
Ann. Vol              0.289122    0.068134  
Ann. IR              -0.0063

In [8]:
ic_best = optimizer.enumerate_optimizer(target_type="return_ic",#优化目标类型
                                        target = "IC Mean",
                                        in_sample_range=None,
                                        ascending=False)
                                        

In [12]:
print(ic_best[0]["signal_name"])
print(ic_best[0]["ic"])
print(ic_best[0]["ret"])
print(ret_best[0]["space"])

divert{'LEN': 3}
                return_ic  upside_ret_ic  downside_ret_ic
IC Mean      3.595960e-02      -0.019193     6.712929e-02
IC Std.      1.016152e-01       0.099141     9.994974e-02
t-stat(IC)   7.769257e+00      -4.250310     1.474531e+01
p-value(IC)  4.811943e-14       0.000026     7.199331e-41
IC Skew     -9.429689e-02      -0.093579     4.525592e-02
IC Kurtosis  2.600698e-02      -0.268392    -2.893212e-01
Ann. IR      3.538800e-01      -0.193596     6.716304e-01
             long_ret   short_ret  long_short_ret  top_quantile_ret  \
t-stat       0.897656    0.718618        1.374921          5.923684   
p-value      0.369810    0.472720        0.169790          0.000000   
skewness    -1.143842    0.914871        0.045719          0.308722   
kurtosis     4.949478    5.284813        9.441826          6.345131   
Ann. Ret     0.047686    0.041418        0.050968          0.074346   
Ann. Vol     0.168335    0.182634        0.117468          0.298537   
Ann. IR      0.283279 

## 自定义Signal优化

In [13]:
close = price.dropna(how='all', axis=1)

In [14]:
import pandas as pd
import talib as ta

signals = dict()
for param in [2,3,4,5]:
    slope_df = pd.DataFrame({name: -ta.LINEARREG_SLOPE(value.values, param) for name, value in close.iteritems()}, index=close.index)
    dv.append_df(slope_df,'slope_param='+str(param))
    signals['slope_param='+str(param)] = optimizer.cal_signal(dv.get_ts('slope_param='+str(param)))
optimizer.all_signals = signals

Nan Data Count (should be zero) : 0;  Percentage of effective data: 68%
Nan Data Count (should be zero) : 0;  Percentage of effective data: 68%
Nan Data Count (should be zero) : 0;  Percentage of effective data: 68%
Nan Data Count (should be zero) : 0;  Percentage of effective data: 67%


In [16]:
ic_best = optimizer.enumerate_optimizer(target_type="return_ic",#优化目标类型
                                        target = "IC Mean",
                                        in_sample_range=None,
                                        ascending=False)                                      

In [17]:
ic_best[0]

{'ic':                 return_ic  upside_ret_ic  downside_ret_ic
 IC Mean      5.305297e-02      -0.011485     9.394080e-02
 IC Std.      1.691483e-01       0.171211     1.645907e-01
 t-stat(IC)   6.857345e+00      -1.466592     1.247851e+01
 p-value(IC)  2.181504e-11       0.143146     4.008065e-31
 IC Skew     -6.268296e-03       0.053573    -1.985105e-01
 IC Kurtosis -5.928649e-02      -0.100609    -6.067405e-02
 Ann. IR      3.136477e-01      -0.067080     5.707539e-01,
 'ret':              long_ret   short_ret  long_short_ret  top_quantile_ret  \
 t-stat       0.737354    1.478470        3.090167          7.462796   
 p-value      0.461270    0.139940        0.002120          0.000000   
 skewness    -0.507883    0.446097        0.838906          0.246972   
 kurtosis     1.735717    2.267040        5.558376          4.441544   
 Ann. Ret     0.043405    0.092156        0.124139          0.099044   
 Ann. Vol     0.185767    0.196703        0.126773          0.314819   
 Ann. IR  